In [1]:
import weaviate 
import os

In [2]:
from dotenv import load_dotenv

load_dotenv()

weaviate_api_key = os.getenv("WEAVIATE_API_kEY")
openai_api_key = os.getenv("OPENAI_API_KEY")



In [7]:
client = weaviate.Client(
    url="https://precision-rag-ov8u665q.weaviate.network", 
    auth_client_secret=weaviate.AuthApiKey(api_key= weaviate_api_key),
     additional_headers={
        "X-OpenAI-Api-Key": openai_api_key  
    }
      )

client.schema.get()

{'classes': []}

In [20]:
client.schema.delete_all()

In [11]:
collection_name = "TenAcademyChallengDocument"

class_obj = {
    "class": collection_name,
    "description": "Represents articles related to AI-driven solutions and language models.",
    "properties": [
        {
            "name": text_content,
            "dataType": ["text"],
            "description": "The title of the article.",
        },
        # {
        #     "name": "content",
        #     "dataType": ["text"],
        #     "description": "The content of the article.",
        # },
    ],
    "vectorizer": "text2vec-openai",  
    "vectorIndexType": "flat",
    "vectorIndexConfig": {
        "bq": {
            "enabled": True, 
            "rescoreLimit": 200,  
            "cache": True,  # e
        },
        "vectorCacheMaxObjects": 100000,  
   "moduleConfig": {
    "generative-openai": {
        "model": "text-davinci-003",  
        "temperature": 0.7, 
        "maxTokens": 200,  
    }
}

    }
}

client.schema.update_config(collection_name, class_obj)

In [12]:
client.schema.get()

{'classes': [{'class': 'TenAcademyChallengDocument',
   'description': 'Represents articles related to AI-driven solutions and language models.',
   'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
    'cleanupIntervalSeconds': 60,
    'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
   'moduleConfig': {'text2vec-openai': {'baseURL': 'https://api.openai.com',
     'model': 'ada',
     'modelVersion': '002',
     'type': 'text',
     'vectorizeClassName': True}},
   'multiTenancyConfig': {'enabled': False},
   'properties': [{'dataType': ['text'],
     'description': 'The title of the article.',
     'indexFilterable': True,
     'indexSearchable': True,
     'moduleConfig': {'text2vec-openai': {'skip': False,
       'vectorizePropertyName': False}},
     'name': 'title',
     'tokenization': 'word'},
    {'dataType': ['text'],
     'description': 'The content of the article.',
     'indexFilterable': True,
     'indexSearchable': True,
     'moduleConfig': {'

In [21]:
with open('context.txt','r') as file:
    text_content = file.read()
    # print(text_content)

In [22]:
collection_name = "TenAcademyChallengDocument"

class_obj = {
    "class": collection_name,
    "description": "Represents articles related to AI-driven solutions and language models.",
    "properties": [
        {
            "name": "content",
            "dataType": ["text"],
            "description": "content",
        },
    ],
    "vectorizer": "text2vec-openai",  
    "vectorIndexType": "flat",
    "vectorIndexConfig": {
        "bq": {
            "enabled": True, 
            "rescoreLimit": 200,  
            "cache": True,  # e
        },
        "vectorCacheMaxObjects": 100000,  
   "moduleConfig": {
    "generative-openai": {
        "model": "text-davinci-003",  
        "temperature": 0.7, 
        "maxTokens": 200,  
    }
}

    }
}

client.schema.create_class(class_obj)

In [43]:
data_object = {
    "class": collection_name,
    "properties": {
        "content": text_content
    }
}


In [44]:
from weaviate.util import generate_uuid5

uuid = client.data_object.create(
    class_name= collection_name,
    data_object= data_object,
    uuid=generate_uuid5(data_object)
)

In [49]:
response = (
    client.query.get(class_name=collection_name, properties=["content"])
    .with_near_text({"concepts": ["bussiness"]})
    .with_limit(3)
    .do()
)
print(response)


{'data': {'Get': {'TenAcademyChallengDocument': [{'content': None}, {'content': 'Business objective  \nPromptlyTech is an innovative e-business specializing in providing AI-driven solutions for optimizing the use of Language Models (LLMs) in various industries. The company aims to revolutionize how businesses interact with LLMs, making the technology more accessible, efficient, and effective. By addressing the challenges of prompt engineering, the company plays a pivotal role in enhancing decision-making, operational efficiency, and customer experience across various industries. PromptlyTech\'s solutions are designed to cater to the evolving needs of a digitally-driven business landscape, where speed and accuracy are key to staying competitive.\nThe company focuses on key services: Automatic Prompt Generation, Automatic Evaluation Data Generation, and Prompt Testing and Ranking.\n1. Automatic Prompt Generation Service:\nThis service streamlines the process of creating effective prompts

In [48]:
import json
json_file_path = 'response3.json'

with open(json_file_path, 'w') as json_file:
    json.dump(response, json_file, indent=2)